#### Classify icebergs and ships on the SAR image

In [8]:
import os

import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from os.path import join
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [12]:
# Define path to the data
PATH = r'C:\Users\arman\Documents\GitHub\sar_ship_detect'
#data_path = join(os. getcwd(), '..','data', 'Iceberg-classifier-challenge')
data_path = join(PATH, 'data', 'Iceberg-classifier-challenge')
train_path = join(data_path, 'train.json', 'train.json')

In [13]:
#Load the data.
train = pd.read_json(train_path)
#test = pd.read_json("../input/test.json")

In [15]:
train.shape

(1604, 5)

In [22]:
#Generate the training data
#Create 3 bands having HH, HV and avg of both
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

data = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],
                          ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [23]:
#Take a look at a iceberg
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):

    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
plotmy3d(X_band_1[12,:,:], 'iceberg')


In [24]:
plotmy3d(X_band_1[14,:,:], 'Ship')

In [58]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.metrics import binary_accuracy

In [26]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

### Train-test split

target=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.75)

In [45]:
# Define the data split strategy
def data_split(data, target, train_size, test_size): 
    ''' Train-validation-test split'''

    # split data to get the initial training test split
    X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=1, 
                                                train_size=train_size, stratify = target)
    
    # split data to get train validation split
    X_train_cv, X_valid, y_train_cv, y_valid = train_test_split( X_train, y_train, 
                                                test_size = valid_size, stratify = y_train) 
    
    return  X_train_cv, X_valid, X_test, y_train_cv, y_valid, y_test

# define split parameters
train_size = 0.90
valid_size = 0.25

X_train_cv, X_valid, X_test, y_train_cv, y_valid, y_test = data_split(data, target, train_size, valid_size )

In [47]:
#Without denoising, core features.
import os
gmodel=getModel()
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=50,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)      

1082/1082 [==============================] - 41s 38ms/step - loss: 0.2672 - accuracy: 0.8826 - val_loss: 0.3140 - val_accuracy: 0.8809
Epoch 36/50
1082/1082 [==============================] - 37s 34ms/step - loss: 0.2619 - accuracy: 0.8697 - val_loss: 0.3360 - val_accuracy: 0.8698
Epoch 37/50
1082/1082 [==============================] - 40s 37ms/step - loss: 0.2231 - accuracy: 0.8909 - val_loss: 0.3060 - val_accuracy: 0.8809
Epoch 38/50
1082/1082 [==============================] - 38s 35ms/step - loss: 0.2740 - accuracy: 0.8780 - val_loss: 0.3700 - val_accuracy: 0.8144
Epoch 39/50
1082/1082 [==============================] - 38s 35ms/step - loss: 0.2653 - accuracy: 0.8734 - val_loss: 0.3324 - val_accuracy: 0.8615


In [49]:
gmodel.load_weights(file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

361/361 [==============================] - 3s 8ms/step
Test loss: 0.3056577816399181
Test accuracy: 0.8753462433815002


#### Predction

In [50]:
predicted_test=gmodel.predict_proba(X_test)

In [66]:
acc = binary_accuracy(y_test.values, predicted_test, threshold=0.5)

In [78]:
acc

<tf.Tensor: id=18727, shape=(161,), dtype=float32, numpy=
array([0.5279503 , 0.47204968, 0.47204968, 0.5279503 , 0.5279503 ,
       0.5279503 , 0.5279503 , 0.5279503 , 0.47204968, 0.47204968,
       0.47204968, 0.47204968, 0.5279503 , 0.47204968, 0.47204968,
       0.47204968, 0.47204968, 0.47204968, 0.47204968, 0.47204968,
       0.5279503 , 0.47204968, 0.5279503 , 0.47204968, 0.5279503 ,
       0.5279503 , 0.47204968, 0.47204968, 0.5279503 , 0.5279503 ,
       0.47204968, 0.5279503 , 0.5279503 , 0.5279503 , 0.5279503 ,
       0.5279503 , 0.47204968, 0.5279503 , 0.5279503 , 0.5279503 ,
       0.47204968, 0.47204968, 0.5279503 , 0.5279503 , 0.5279503 ,
       0.47204968, 0.47204968, 0.5279503 , 0.5279503 , 0.5279503 ,
       0.5279503 , 0.5279503 , 0.47204968, 0.5279503 , 0.47204968,
       0.5279503 , 0.5279503 , 0.5279503 , 0.47204968, 0.5279503 ,
       0.47204968, 0.5279503 , 0.5279503 , 0.47204968, 0.5279503 ,
       0.5279503 , 0.47204968, 0.47204968, 0.47204968, 0.47204968,
    